In [1]:
import sys
sys.path.append("..")

In [2]:
import numpy as np
import pandas as pd
from arcgis.gis import GIS
from arcgis.geocoding import geocode, batch_geocode
from ZillowUnivariate.univ_stats import SingleTableReader

In [3]:
gis = GIS(username='u1singh_UCSDOnline9', password='Udaikaran98')

In [4]:
file_dir = '/Volumes/GoogleDrive/Shared drives/Zillow1/data/raw/ZTrans_ZAsmt/unzipped_20191001/02'

In [5]:
columnsToUse = ['PropertyFullStreetAddress', 
                'PropertyCity','PropertyState','PropertyZip']

In [6]:
prop_table = SingleTableReader("ZTrans", "PropertyInfo", file_dir).read(nrows = 5000, usecols = columnsToUse)

In [7]:
def define_address(row):
    state = row['PropertyState']
    zipCode = row['PropertyZip']
    city = row['PropertyCity']
    address = row['PropertyFullStreetAddress']
    if not pd.isna(state):
        if not pd.isna(zipCode):
            if not pd.isna(city):
                if not pd.isna(address):
                    return address + ',' + city + ',' + state + ',' + zipCode
                else:
                    return city + ',' + state + ',' + zipCode
            else:
                return state + ',' + zipCode
        else:
            return state
    else:
        return np.nan

In [8]:
prop_table['fixed_address'] = prop_table.apply(define_address, axis = 1)

In [9]:
addresses = list(prop_table.fixed_address)

In [11]:
results = batch_geocode(addresses)

Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (sin

Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (sin

Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (sin

address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dict

address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dictionary (with address fields as keys)
Unsupported address: nan
address should be a string (single line address) or dict

In [12]:
df = pd.DataFrame(columns = ['score', 'latitude', 'longitude', 'address'])
count = 0
for res in results:
    if (res == None):
        row = [np.nan, np.nan, np.nan, np.nan]
    elif (res['score'] == 0):
        row = [np.nan, np.nan, np.nan, np.nan]
    else:
        row = [res['score'], res['attributes']['X'],
               res['attributes']['Y'], res['attributes']['Place_addr']]
    df.loc[len(df)] = row

In [13]:
fin_table = pd.concat([prop_table,df], axis = 1)

In [15]:
fin_table.to_csv("sample_geocoding.csv")

In [ ]:
import os
from pathlib import Path
from subprocess import Popen
import boto3
import zipfile

In [ ]:
s3 = boto3.client('s3')

In [ ]:
download_bucket = "temp"
upload_bucket = "output"

TMPDIR = Path("zillow-data-raw/20191001/")
FIPS_table = ['2']

In [ ]:
FipsFile = FIPS_table[0] + '.zip'
Key = '20191001/' + FipsFile
Filename = str(TMPDIR / FipsFile)
print(Filename)

In [ ]:
# download zip file from s3 to ec2
s3.download_file(
    Bucket=download_bucket,
    Key=Key,
    Filename=Filename
)